In [ ]:
import requests
import os
import json
import time

def processQueue():
    for channel in queue:
        try:
            channel_id = str(channel['id'])
                    
            username = str(channel['user']['username'])
            if username:
                if os.path.isdir('d2-stream-name-parser/' + username) == False:
                    os.mkdir('d2-stream-name-parser/' + username)
                data = {}
                try:
                    with open('d2-stream-name-parser/' + username + '/mixer.json') as f:
                        for i in f:
                            data = json.loads(i)
                except:
                    pass
                
                if channel_id in data:
                    data[channel_id] += 1
                else:
                    data[channel_id] = 1
                with open('d2-stream-name-parser/' + username + '/mixer.json', 'w+') as f:
                    f.write(json.dumps(data))

        except:
            pass

def getStreams(offset = 0):
    mixerBaseurl = 'https://mixer.com/api/v1/'
    clientID = '70eaab0506c7699b2c1800b9ce485786273c5db3b65d80c9'
    headers = {'Client-ID': clientID}
    
    streamsUrl = mixerBaseurl + 'channels?limit=100&where=typeId:eq:543113&page=' + str(offset)
    r = requests.get(streamsUrl, headers = headers)
    json = r.json()
    if json:
        for channel in json:
            if ('recov' not in channel['name']):
                queue.append(channel)
        getStreams(offset + 1)
    else:
        processQueue()

queue = []

try:
    while True:
        queue = []
        
        print('getting screens...')
        try:
            getStreams()
        except:
            pass
                
        print('committing...')
        !git -C d2-stream-name-parser add .
        !git -C d2-stream-name-parser commit -m 'add mixerids'
        
        print('sleeping...')
        time.sleep(900)
except KeyboardInterrupt:
    print('Stopped by user')